In [129]:
 current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"


import os
import glob
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import re
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    RidgeCV,
    LassoCV,
    ElasticNetCV,
)


# Mount Google Drive
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
os.chdir(current_dir)

In [4]:
import pandas as pd
copy = pd.read_pickle("pos_dataframe.pkl")

#Experiments
##Train: handbook
##Validate: doc
##Test: cbc & article

In [5]:
columns_to_keep  = ['ADJ', 'ADP', 'ADV', 'AUX', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'SCONJ', 'VERB', 'group', 'name', "score"]

In [96]:
df = copy[columns_to_keep]
# Split the DataFrame into 'train' and 'validate' based on the 'group' column
handbook = df[df['group'] == 'handbook']
doc = df[df['group'] == 'doc']
cbc = df[df['group'] == 'cbc']
other = df[~df['group'].isin(['handbook', 'doc', 'cbc'])]

In [122]:
train = pd.concat([cbc, doc, other], ignore_index=True)
validate = handbook

In [123]:
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    SVR(),
]
features = ['ADJ', 'ADP', 'ADV', 'AUX', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'SCONJ', 'VERB',]

In [124]:
from itertools import combinations

features = ['ADJ', 'ADP', 'ADV', 'AUX', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'SCONJ', 'VERB']
feature_combos = [list(combo) for combo in combinations(features, 2)]

In [125]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

def create_and_train_model(train_data, feature_columns, model):
    loo = LeaveOneOut()
    y_preds, y_trues = [], []

    # Create and fit CountVectorizers outside the loop
    vectorizers = {col: CountVectorizer().fit(train_data[col]) for col in feature_columns}

    # Create the ColumnTransformer with fitted CountVectorizers
    transformers = [(col, vectorizers[col], col) for col in feature_columns]
    preprocessor = ColumnTransformer(transformers=transformers, remainder='drop')

    for train_index, test_index in loo.split(train_data):
        X_train, X_test = train_data.iloc[train_index], train_data.iloc[test_index]
        y_train, y_test = train_data['score'].iloc[train_index], train_data['score'].iloc[test_index]

        # Create a pipeline with preprocessor and model
        pipe = Pipeline([('preprocessor', preprocessor), ('model', model)])
        # Fit the pipeline on the training data
        pipe.fit(X_train[feature_columns], y_train)
        y_pred = pipe.predict(X_test[feature_columns])

        # Extract single prediction and actual value
        y_pred_single = y_pred[0]
        y_preds.append(y_pred)
        y_trues.append(y_test.iloc[0])


    metrics = {
        'R²': r2_score(y_trues, y_preds),
        'MSE': mean_squared_error(y_trues, y_preds),
        'RMSE': np.sqrt(mean_squared_error(y_trues, y_preds)),
        'MAE': mean_absolute_error(y_trues, y_preds)
    }

    return {
        'Trained Pipeline': pipe,
        'Model': str(model),
        'Feature Columns': feature_columns,
        'Evaluation Metrics': metrics
    }


In [126]:
def validate_trained_pipe(val_data, pipeline_info):
    model_name = pipeline_info["Model"]
    features = pipeline_info["Feature Columns"]
    pipeline = pipeline_info["Trained Pipeline"]
    # Predict using the pipeline
    y_pred = pipeline.predict(val_data)
    y_true = val_data['score']
    # Calculate evaluation metrics
    metrics = {
        'R²': r2_score(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }
    # Print Names, Actual Scores, and Predicted Scores
    print("\n")
    print(f"model: {model} features: {features}")
    for name, actual, predicted in zip(val_data['name'], y_true, y_pred):
        print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")
    return metrics






In [131]:
for feature_combo in feature_combos:
  for model in models:
    pipe_data = create_and_train_model(train, feature_combo, model)
    validation_data = validate_trained_pipe(validate, pipe_data)
    print(validation_data)





model: LinearRegression() features: ['ADJ', 'ADP']
Name: bundy, Actual Score: 34.0, Predicted Score: 42.463074608863266
Name: dahmer, Actual Score: 23.0, Predicted Score: 39.3631887307222
Name: gacy, Actual Score: 27.0, Predicted Score: 37.428646427546646
Name: kemper, Actual Score: 26.0, Predicted Score: 38.13778129750605
Name: ridgeway, Actual Score: 19.0, Predicted Score: 38.117639539102214
{'R²': -6.825277849807275, 'MSE': 192.1888239912667, 'RMSE': 13.8632183850384, 'MAE': 13.302066120748076}


model: Ridge() features: ['ADJ', 'ADP']
Name: bundy, Actual Score: 34.0, Predicted Score: 42.43458801789587
Name: dahmer, Actual Score: 23.0, Predicted Score: 39.347270112993314
Name: gacy, Actual Score: 27.0, Predicted Score: 37.42083748310747
Name: kemper, Actual Score: 26.0, Predicted Score: 38.127571830722076
Name: ridgeway, Actual Score: 19.0, Predicted Score: 38.10802606342326
{'R²': -6.810781983533557, 'MSE': 191.8328055155842, 'RMSE': 13.850372035277038, 'MAE': 13.287658701628397}

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.705e-03, tolerance: 1.869e-03
  model = cd_fast.enet_coordinate_descent(




model: Lasso() features: ['AUX', 'PRON']
Name: bundy, Actual Score: 34.0, Predicted Score: 38.8378850777745
Name: dahmer, Actual Score: 23.0, Predicted Score: 37.5868564712754
Name: gacy, Actual Score: 27.0, Predicted Score: 39.9382292444199
Name: kemper, Actual Score: 26.0, Predicted Score: 36.873678146703256
Name: ridgeway, Actual Score: 19.0, Predicted Score: 35.64050476085995
{'R²': -5.504255414127027, 'MSE': 159.7445129709598, 'RMSE': 12.639007594386507, 'MAE': 11.975430740206601}


model: ElasticNet() features: ['AUX', 'PRON']
Name: bundy, Actual Score: 34.0, Predicted Score: 37.46579621663423
Name: dahmer, Actual Score: 23.0, Predicted Score: 35.64619889793872
Name: gacy, Actual Score: 27.0, Predicted Score: 39.55852563110781
Name: kemper, Actual Score: 26.0, Predicted Score: 35.748782603178526
Name: ridgeway, Actual Score: 19.0, Predicted Score: 34.93015978211159
{'R²': -4.5249463268428025, 'MSE': 135.69268178725923, 'RMSE': 11.648720178082193, 'MAE': 10.869892626194176}


mo